# **Chapter 4: Separating Data from Instructions**

---
**Lesson:**

Oftentimes, we don't want to write full prompts, but instead want prompt templates that can be modified later with additional input data before submitting to Claude. This might come in handy if you want Claude to do the same thing every time, but the data that Claude uses for its task might be different each time. 

Luckily, we can do this pretty easily by separating the fixed skeleton of the prompt from variable user input, then substituting the user input into the prompt before sending the full prompt to Claude. 

Below, we'll walk step by step through how to write a substitutable prompt template, as well as how to substitute in user input.

---

First we will setup our dependencies

In [3]:
%%capture
#Cell [1]

#langchain
%pip install langchain

#Install libraries 
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

#Import libraries
import json
import os
import sys
import boto3
import botocore

#Set up Bedrock runtime client
bedrock_runtime = boto3.client('bedrock-runtime')
boto3_bedrock = boto3.client('bedrock')

#Set up ModelID (Claude 3 Sonnet in this case)
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

In [4]:
#Cell [2]

#Setup Model invocation function 
#Code modified from documentation: https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_Claude3_Text_section.html
def get_response(prompt_data):
    response = bedrock_runtime.invoke_model(
                    modelId=modelId,
                    body=json.dumps(
                        {
                            "anthropic_version": "bedrock-2023-05-31",
                            "max_tokens": 1024,
                            "messages": [
                                {
                                    "role": "user",
                                    "content": [{"type": "text", "text": prompt_data}],
                                }
                            ],
                        }
                    ),
                )
    result = json.loads(response.get("body").read())
    response = result.get("content", [])[0]['text']
    return response

# **Examples:**

In this first example (4.1), we're asking Claude to act as an animal noise generator. Notice that the full prompt submitted to Claude is just the prompt template substituted with the input (in this case, "Cow"). Notice that the word "Cow" replaces the "{ANIMAL}" section.

> *Note: You don't have to call your substitution placeholder anything in particular in practice. For this example, definitely use {ANIMAL}, as that's how the exercise is formatted. But in general, just as easily, we could have called it "{CREATURE}" or "{A}" (but it's generally good to have your stand-ins be specific and relevant so that your prompt is easy to understand even without the substitution). Just make sure that whatever you name your substitution placeholder is what you use for the substitution formula."*

**Example 4.1 - Moo**

In [8]:
from langchain import PromptTemplate
# Create a prompt template that has input variables
multi_var_prompt = PromptTemplate(
    input_variables=["ANIMAL"], 
    template="""

Human: I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}.

Assistant:"""
)

# Pass in values to the input variables
prompt = multi_var_prompt.format(ANIMAL="Cow")

response = get_response(prompt)

print(response)

Moo.


**Example 4.2 - This could have been an email**

Why would we want to separate and substitute inputs like this? Well, prompt templates simplify repetitive tasks. Let's say you build a prompt structure that invites third party users to submit content to the prompt (in the previous example the animal whose sound they want to generate). These third party users don't have to write or even see the full prompt. All they have to do is fill in variables.

We do this substitution here leverage an open source Python library called Langchain.  We use the {curly-brackets} formatting in our own code.

> *Note: Prompt templates can have as many variables as desired.*

When introducing substitution variables like this, it is very important to make sure Claude knows where variables start and end (vs. instructions or task descriptions). Let's look at an example where there is no separation between the instructions and the substitution variable.		

> *Note: We recommend that you use specifically XML tags as separators for Claude, as opposed to some other separator, as Claude's already trained to recognize XML tags in particular."*		

In [9]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], 
    template="""

Human: Hey Claude. 

<email> {EMAIL} </email>

Make this email more polite.

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="Show up at 6am because I'm the CEO and I say so")

response = get_response(prompt)

print(response)

Here's a more polite version of the email:

<email>
Subject: Early Morning Meeting Request
Dear Team,

I hope this email finds you well. I wanted to request an early morning meeting at 6 AM to discuss some important matters. As the CEO, I understand the inconvenience this may cause, and I appreciate your flexibility and dedication.

Please let me know if you have any conflicts or concerns, and we can try to accommodate them as best as possible. Your presence and input are highly valued, and I look forward to a productive discussion.

Thank you for your understanding and cooperation.

Best regards,
[Your Name]
CEO
</email>


**Example 4.3 - The farm**

Let's see another example of how XML tags can help us. 
We need to surround the user input sentences in XML tags. This shows Claude where the input data begins and ends despite the misleading hyphen before "Each is about an animal, like rabbits."	

In [10]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["SENTENCES"], 
    template="""

Human: I will give you a list of sentences. 

-Each is about an animal, like rabbits.
<sentences>
{SENTENCES}
</sentences>
Tell me the second item on the list. 

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(SENTENCES="""
-I like how cows sound
-This sentence is about spiders
-This sentence may appear to be about dogs but it's actually about pigs""")

response = get_response(prompt)

print(response)

The second item on the list is "This sentence is about spiders".


# **Exercises**

The following exercises will need you to manipulate the prompt to get the desired output

**Excercise 4.1 - Haiku Topic**

Write a prompt in the highlighted template box that will take in a variable called "{TOPIC}" and output a haiku about the topic.

Remember to format everything properly, the way we covered in earlier lessons.

In [11]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["TOPIC"], 
    template="""

Human: YOUR PROMPT HERE  

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(TOPIC="YOUR TOPIC HERE")

response = get_response(prompt)

print(response)

I'm afraid I don't have a specific prompt to respond to since you didn't provide one. I'm Claude, an AI assistant created by Anthropic to be helpful, harmless, and honest. Please feel free to ask me any question or provide a prompt, and I'll do my best to provide a relevant and insightful response.


**Exercise 4.2 - Dog Question With Typos**

Fix the prompt in the template below by adding XML tags, or making small edits so that Claude produces the right answer.

Try not to change anything else about the prompt. The messy and mistake-ridden writing is intentional, so you can see how Claude reacts to such mistakes.

In [12]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["QUESTION"], 
    template="""

Human: hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx  

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(QUESTION="r dogs brownw?")

response = get_response(prompt)

print(response)

I'm having a bit of trouble understanding your question due to the jumbled words and letters. Could you please rephrase your question about dogs more clearly? I'd be happy to try to provide a short answer once I can make out what you're asking.


# Chapter 4 - END.